In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import scipy.stats as scs
%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid", palette="pastel", color_codes=True)

import sys

print(sys.stdout.encoding)

UTF-8


Data Source:

https://www.kaggle.com/kemical/kickstarter-projects

## Still some more EDA stuff

** In order to run this notebook, you must first navigate to the src folder within the github directory and run the PYTHON file extract_features.py** 

**This will create the file: '"data_with_dummies_and_class.csv" using "imputed.csv" as a basis**
> **IT TAKES ABOUT 15 MIN TO RUN extract_features.py **

In [77]:
try:
    data= pd.read_csv('data/data_with_dummies_and_class.csv')
except FileNotFoundError:
    data = pd.read_csv("https://s3-us-west-2.amazonaws.com/twisk.us/data_with_dummies_and_class.csv")
    data.to_csv('data/data_with_dummies_and_class.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Columns: 257 entries, id to launched_month&Jun
dtypes: bool(236), float64(15), int64(6)
memory usage: 145.9 MB


In [ ]:
data = data[[x for x in data.columns if "Unnamed" not in x]]

In [ ]:
unique_cols = list(set(x.split('&')[0] for x in data.columns))

In [ ]:
df_nocats = data[[x for x in data.columns if x.split('&')[0] != 'category']]

In [ ]:
def p_func(df):
    ooot = {}
    for col in df.columns:
        if '$' in col:
            value = ooot.get(col.split('$')[0], [])
            ooot[col.split('$')[0]] = value + [col.split('$')[1]]
        elif '&' in col:
            value = ooot.get(col.split('&')[0], [])
            ooot[col.split('&')[0]] = value + [col.split('&')[1]]
        else: 
            ooot[col] = ''
    return ooot

In [ ]:
for k,v in p_func(df_nocats).items():
    print(k)
    if type(v) is list:
        for i in v:
            print('\t>', i)

In [94]:
dff = df_nocats.drop('''country&N,0"''',axis=1)  ##.dropna()    name$ all have 1 na

name = ['name$word_count','name$punc_count','name$caps_count','name$possesive_count','name$noun_count',
       'name$adj_count','name$verb_count','name$preposition_count','name$determinator_count']
main_cat = ['main_category&Publishing','main_category&Film & Video','main_category&Music',
           'main_category&Food','main_category&Design','main_category&Crafts','main_category&Games',
           'main_category&Comics','main_category&Fashion','main_category&Theater','main_category&Art',
           'main_category&Photography','main_category&Technology','main_category&Dance','main_category&Journalism']

dff_name = dff[name]
dff_main_cat = dff[main_cat]

In [44]:
main_features = p_func(dff).keys()
print(len(main_features))

19


In [45]:
from sklearn import linear_model as lm
from sklearn import svm as svm

In [46]:
## creating arrays for train/test set indices
train_idx = np.random.random(dff.shape[0]) < .8    
test_idx = train_idx == False

In [47]:
## train and test y vector
ytr = dff.iloc[train_idx].pop('success')      
yte = dff.iloc[test_idx].pop('success')
y = dff.pop('success')

In [48]:
## train and test X matrix
Xtr = dff.iloc[train_idx][[col for col in dff.columns if  col != 'success']]
Xte = dff.iloc[test_idx][[col for col in dff.columns if  col != 'success']]
X = dff[[col for col in dff.columns if  col != 'success']]

In [55]:
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

lr_clf.fit(Xtr,ytr)  # train object
y_hat = lr_clf.predict(Xte) # get test set precitions

acc = mt.accuracy_score(yte,y_hat)
conf = mt.confusion_matrix(yte,y_hat)
print("accuracy", acc )
print("confusion matrix\n",conf)

accuracy 0.6421290305543085
confusion matrix
 [[48292     0]
 [27103   339]]


In [105]:
##testing it w/o any categorical variables

data_imp = pd.read_csv('data/imputed.csv')
cat_column = ['name','category','currency','deadline','launched','state','country',"main_category"]

data_imp.drop(cat_column, inplace=True, axis=1)

data_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 13 columns):
id                      378661 non-null int64
backers                 378661 non-null int64
usd_pledged_real        378661 non-null float64
usd_goal_real           378661 non-null float64
log_goal                378661 non-null float64
duration                378661 non-null int64
launched_month          378661 non-null int64
success                 378661 non-null int64
log_backers             378661 non-null float64
log_usd_goal_real       378661 non-null float64
log_usd_pledged_real    378661 non-null float64
log_duration            378661 non-null float64
isUS                    378661 non-null int64
dtypes: float64(7), int64(6)
memory usage: 37.6 MB


In [87]:
## printing accuracy % and confusion matrix for different dfs

def log_regression(df):
    train_idx = np.random.random(df.shape[0]) < .8    
    test_idx = train_idx == False

    ytr = df.iloc[train_idx].pop('success')
    yte = df.iloc[test_idx].pop('success')

    Xtr = df.iloc[train_idx][[col for col in df.columns if  col != 'success']]
    Xte = df.iloc[test_idx][[col for col in df.columns if  col != 'success']]

    lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

    lr_clf.fit(Xtr,ytr)  # train object
    y_hat = lr_clf.predict(Xte) # get test set precitions

    acc = mt.accuracy_score(yte,y_hat)
    conf = mt.confusion_matrix(yte,y_hat)
    print("accuracy", acc )
    print("confusion matrix\n",conf)


In [88]:
log_regression(data_imp)

## marginally better, could be due to randomness

accuracy 0.6484683924711341
confusion matrix
 [[48887     5]
 [26665   311]]


In [97]:
## all numerical variables + word information

df = pd.concat([data_imp,dff_name]).dropna()

log_regression(data_imp)

## marginally better

accuracy 0.6501347566453523
confusion matrix
 [[48886     6]
 [26476   324]]


In [98]:
## all numerical variables + main_cat or type of project

df = pd.concat([data_imp,dff_main_cat]).dropna()

log_regression(data_imp)

## very marginally better than last

accuracy 0.6504231144386154
confusion matrix
 [[49009     3]
 [26518   336]]


In [99]:
## all numerical + name + main_cat

df = pd.concat([data_imp,dff_name,dff_main_cat]).dropna()

log_regression(data_imp)

## marginally worse

accuracy 0.648114948589507
confusion matrix
 [[48868     4]
 [26690   298]]


In [118]:
## using a model that has just the goal vs pledged obviously will yeild great results

df_basic = data_imp[['log_usd_goal_real','log_usd_pledged_real','success']]

log_regression(df_basic)

accuracy 0.9855353120183378
confusion matrix
 [[47799  1098]
 [    0 27012]]


In [119]:
## using the model df_basic model
df = df_basic

train_idx = np.random.random(df.shape[0]) < .8    
test_idx = train_idx == False

ytr = df.iloc[train_idx].pop('success')
yte = df.iloc[test_idx].pop('success')

Xtr = df.iloc[train_idx][[col for col in df.columns if  col != 'success']]
Xte = df.iloc[test_idx][[col for col in df.columns if  col != 'success']]

lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

lr_clf.fit(Xtr,ytr)

# iterate over the coefficients
weights = lr_clf.coef_.T # take transpose to make a column vector
variable_names = df.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])
    
# does this look correct? 

## We'd assume the lower the goal, the easier a 'success' is, so a negative coefficient makes sense
## opposite is true for pledged
## equal weights for the 2 makes sense

log_usd_goal_real has weight of -15.55734254173411
log_usd_pledged_real has weight of 15.709142662452726


In [121]:
from sklearn.preprocessing import StandardScaler

# we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)
# to Make things easier, let's start by just using whatever was last stored in the variables:
##    X_train , y_train , X_test, y_test (they were set in a for loop above)

# scale attributes by the training set
scl_obj = StandardScaler()
scl_obj.fit(Xtr) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data

X_train_scaled = scl_obj.transform(Xtr) # apply to training
X_test_scaled = scl_obj.transform(Xte) # apply those means and std to the test set (without snooping at the test set values)

# train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=0.05) # get object, the 'C' value is less (can you guess why??)
lr_clf.fit(X_train_scaled,ytr)  # train object

y_hat = lr_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(yte,y_hat)
conf = mt.confusion_matrix(yte,y_hat)
print('accuracy:', acc )
print(conf )

# sort these attributes and spit them out
zip_vars = zip(lr_clf.coef_.T,df.columns) # combine attributes
zip_vars = sorted(zip_vars)
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0]) # now print them out

accuracy: 0.981988738027576
[[47740  1369]
 [    0 26899]]
log_usd_goal_real has weight of -7.6746497280652815
log_usd_pledged_real has weight of 15.165573874459227


In [123]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


weights = pd.Series(lr_clf.coef_[0],index=df.columns)
weights.plot(kind='bar')
plt.show()

ValueError: Wrong number of items passed 2, placement implies 3

In [100]:
from sklearn.model_selection import ShuffleSplit

num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


In [79]:
## professor's code -- doesn't work.

from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

KeyError: '[155065 326449 289422 ... 341401 213692 192962] not in index'

In [31]:
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    lr_clf.fit(X[train_indices],y[train_indices])  # train object
    y_hat = lr_clf.predict(X[test_indices]) # get test set precitions

    # print the accuracy and confusion matrix 
    print("====Iteration",iter_num," ====")
    print("accuracy", mt.accuracy_score(y[test_indices],y_hat)) 
    print("confusion matrix\n",mt.confusion_matrix(y[test_indices],y_hat))

KeyError: '[295845  84084 198964 ... 331681 193339 277992] not in index'

In [39]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(lr_clf, X, y=y, cv=cv_object) # this also can help with parallelism
print(accuracies)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [56]:
from ipywidgets import widgets as wd

def lr_explor(cost):
    lr_clf = LogisticRegression(penalty='l2', C=cost, class_weight=None) # get object
    accuracies = cross_val_score(lr_clf,X,y=y,cv=cv_object) # this also can help with parallelism
    print(accuracies)

wd.interact(lr_explor,cost=(0.001,5.0,0.05),__manual=True)

[0.64293826 0.64304389 0.6436645 ]


<function __main__.lr_explor>